In [1]:

!pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd


df = pd.read_csv('CAvideos.csv')
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [6]:
import pandas as pd
import json

def combine_features(row):
    try:
        return row['title'] + " " + row["channel_title"]
    except Exception as e:
        print("Error:", row, e)

def process_us_videos_csv(input_file, output_file):
    # Load the CSV file
    videos = pd.read_csv(input_file)
    
    # Fill missing values for relevant columns
    for f in ['title', 'channel_title']:
        videos[f] = videos[f].fillna('')
    
    # Create a "text" column by combining title and channel_title
    videos["text"] = videos.apply(combine_features, axis=1)
    
    # Select and rename columns to match required Vespa format
    # videos = videos[['video_id', 'title', 'text', 'views', 'likes', 'dislikes', 'comment_count', 'publish_time']]
    videos = videos[['video_id', 'title', 'text']]
    videos.rename(columns={'video_id': 'doc_id'}, inplace=True)
    
    # Create 'fields' column as JSON-like structure of each record
    videos['fields'] = videos.apply(lambda row: row.to_dict(), axis=1)
    
    # Create 'put' column based on 'doc_id'
    videos['put'] = videos['doc_id'].apply(lambda x: f"id:hybrid-search:doc::{x}")
    
    # Prepare the final DataFrame for JSONL export
    df_result = videos[['put', 'fields']]
    
    # Output the processed data to a JSON file in Vespa-compatible format
    df_result.to_json(output_file, orient='records', lines=True)


# Usage example
process_us_videos_csv("CAvideos.csv", "clean_ca_videos.jsonl")


In [4]:
!pip3 install --ignore-installed vespacli

  Using cached vespacli-8.391.23-py3-none-any.whl.metadata (15 kB)
Using cached vespacli-8.391.23-py3-none-any.whl (47.7 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!vespa config set target local

In [3]:
!vespa status deploy --wait 300

Waiting up to 5m0s for deploy API...
Deploy API at http://127.0.0.1:19071 is ready


In [4]:
!vespa deploy --wait 300 app

Waiting up to 5m0s for deploy API...
Uploading application package... done;1m⣻

Success: Deployed 'app' with session ID 4
WARNING File in application package with unknown extension: schemas/.DS_Store, please delete or move file to another directory.
Waiting up to 5m0s for deployment to converge...
Waiting up to 5m0s for cluster discovery...
Waiting up to 5m0s for container default...


In [7]:

!vespa feed -t http://localhost:8080 clean_ca_videos.jsonl

{
  "feeder.operation.count": 40881,
  "feeder.seconds": 166.465,
  "feeder.ok.count": 40881,
  "feeder.ok.rate": 245.582,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 40881,
  "http.request.bytes": 7789145,
  "http.request.MBps": 0.047,
  "http.exception.count": 0,
  "http.response.count": 40881,
  "http.response.bytes": 4169862,
  "http.response.MBps": 0.025,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 18,
  "http.response.latency.millis.avg": 91,
  "http.response.latency.millis.max": 598,
  "http.response.code.counts": {
    "200": 40881
  }
}


In [ ]:
vespa stat

: 

In [ ]:
!vespa feed -t http://localhost:8080 clean_ca_videos.jsonl

: 

In [8]:
!pip install pyvespa

  Using cached pyvespa-0.50.0-py3-none-any.whl.metadata (18 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached cryptography-43.0.3-cp39-abi3-macosx_10_9_universal2.whl.metadata (5.4 kB)
  Using cached aiohttp-3.10.10-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached fastcore-1.7.19-py3-none-any.whl.metadata (3.5 kB)
  Using cached lxml-5.3.0-cp313-cp313-macosx_10_13_universal2.whl.metadata (3.8 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  

In [9]:
# Import necessary libraries
import pandas as pd
from vespa.application import Vespa
from vespa.io import VespaResponse, VespaQueryResponse

# Function to display hits as a DataFrame
def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"].get(field, None)
        records.append(record)
    return pd.DataFrame(records)

# Keyword search function using the BM25 ranking profile
def keyword_search(app, search_query):
    query = {
        "yql": "select * from sources * where userQuery() limit 5",
        "query": search_query,
        "ranking": "bm25",
    }
    response = app.query(query)
    return display_hits_as_df(response, ["doc_id", "title", "text"])

# Semantic search function using embedding
def semantic_search(app, query):
    query = {
        "yql": "select * from sources * where ({targetHits:100}nearestNeighbor(embedding,e)) limit 5",
        "query": query,
        "ranking": "semantic",
        "input.query(e)": "embed(@query)"
    }
    response = app.query(query)
    return display_hits_as_df(response, ["doc_id", "title", "text"])

# Retrieve embedding for a specific doc_id
def get_embedding(doc_id):
    query = {
        "yql" : f"select doc_id, title, text, embedding from doc where doc_id contains \'{doc_id}\'",
        "hits": 1
    }
    result = app.query(query)
    
    if result.hits:
        return result.hits[0]
    return None

# Query for recommendations based on embedding
def query_videos_by_embedding(embedding_vector):
    query = {
        'hits': 5,
        'yql': 'select * from doc where ({targetHits:5}nearestNeighbor(embedding, user_embedding))',
        'ranking.features.query(user_embedding)': str(embedding_vector),
        'ranking.profile': 'recommendation'
    }
    return app.query(query)

# Replace with the host and port of your local Vespa instance
app = Vespa(url="http://localhost", port=8080)

# Sample queries for testing
query = "Freakonomics"

# Perform keyword search
df = keyword_search(app, query)
print(df.head())

# Perform semantic search
df = semantic_search(app, query)
print(df.head())

# Retrieve embedding for a specific doc ID and use it for recommendations
emb = get_embedding("2kyS6SvSYSE")  # Sample doc_id
if emb:
    results = query_videos_by_embedding(emb["fields"]["embedding"])
    df = display_hits_as_df(results, ["doc_id", "title", "text"])
    print(df.head())

  doc_id         title                                               text
0  43942  Freakonomics  Some of the world's most innovative documentar...
        doc_id                                              title  \
0        43942                                       Freakonomics   
1         7512                                          Idiocracy   
2  wCcjEszzrTI  3.20 - Podesta to Guantanamo?/Facebook Implodi...   
3  lPuwEd2PMUQ   *NEW* PUBLIC FREAKOUTS & CRINGE INTERACTIONS #81   
4  1XHb4zHzz30  IT'S HAPPENING!! Peter Strzok FLIPS ON OBAMA &...   

                                                text  
0  Some of the world's most innovative documentar...  
1  To test its top-secret Human Hibernation Proje...  
2  3.20 - Podesta to Guantanamo?/Facebook Implodi...  
3  *NEW* PUBLIC FREAKOUTS & CRINGE INTERACTIONS #...  
4  IT'S HAPPENING!! Peter Strzok FLIPS ON OBAMA &...  
        doc_id                                              title  \
0  2kyS6SvSYSE                 WE WANT